## Step 0: Imports and Reading Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mtick
import matplotlib.lines as lines
import matplotlib.image as mpimg
import matplotlib.colors as mcolors
import seaborn as sns
import ast

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("10000 Movies Data.csv")

## Step 0.1: Design the Charts


#### General Chart Design

In [ ]:
def chartStyling(ax, fig, title, bars, logo_path='logo.png'):
    fig.patch.set_facecolor('#D3D3D3')
    ax.set_facecolor('#D3D3D3')

    fig.text(0.09, 1.05, title, fontsize=18, fontweight='bold', fontfamily='serif')

    logo = mpimg.imread(logo_path)
    logo_ax = fig.add_axes([0.85, 0.85, 0.1, 0.1])
    logo_ax.imshow(logo)
    logo_ax.axis('off')

    ax.grid(axis='y', linestyle='-', alpha=0.2)
    ax.set_axisbelow(True)

    for spine in ['top', 'right', 'left']:
        ax.spines[spine].set_visible(False)

    ax.tick_params(axis='both', which='major', labelsize=12)

    l1 = lines.Line2D([1, 1], [0, 1], transform=fig.transFigure, figure=fig, color='black', lw=0.2)
    fig.lines.extend([l1])

    for bar in bars:
        bar.set_color('#008080') 
        
    for bar in bars:
        bar.set_edgecolor('#000000')
        bar.set_linewidth(0.5)

    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

### Pie Charts Design

In [ ]:
def pieChartStyling(values, labels, title, figsize=(18, 10)):
    fig, ax = plt.subplots(figsize=figsize)
    
    total = sum(values)
    percentages = [v/total*100 for v in values]
    
    wedges, texts, autotexts = ax.pie(values,
                                     labels=labels,
                                     autopct='%1.1f%%',
                                     startangle=140,
                                     colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'])
    
    fig.patch.set_facecolor('#f0f0f0')
    ax.set_facecolor('#f0f0f0')
    
    fig.text(0.5, 0.95, title, ha='center', fontsize=18, fontweight='bold', fontfamily='serif')
    
    for autotext in autotexts:
        autotext.set_color('black')
        autotext.set_fontsize(12)
        autotext.set_fontweight('bold')
    
    ax.legend([f'{l} ({p:.1f}%)' for l, p in zip(labels, percentages)], 
             loc='center left', 
             bbox_to_anchor=(1, 0.5), 
             fontsize=12)
    
    plt.show()

### Line Chart Design

In [ ]:

def lineChartStyling(ax, fig, title):
    fig.patch.set_facecolor('#D3D3D3')
    ax.set_facecolor('#D3D3D3')
    
    fig.text(0.09, 1.05, title, fontsize=18, fontweight='bold', fontfamily='serif')
    
    ax.grid(axis='y', linestyle='-', alpha=0.2)
    ax.set_axisbelow(True)
    
    for spine in ['top', 'right', 'left']:
        ax.spines[spine].set_visible(False)
        
    ax.tick_params(axis='both', which='major', labelsize=12)
    
    l1 = lines.Line2D([1, 1], [0, 1], transform=fig.transFigure, figure=fig, color='black', lw=0.2)
    fig.lines.extend([l1])



## Step 1: Data Understanding

In [ ]:
df.head(5)

In [ ]:
print(df.shape[0])
print(df.shape[1])

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
missingData = df.isnull().sum().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(18, 10))

bars = ax.bar(missingData.index, missingData.values)

chartStyling(fig= fig, ax = ax, bars= bars, title = "Missing Values")

plt.show()

In [ ]:
for column in df.columns:
    print(f"{column} : {len(df[column].unique())}")

## Step 2: Data Preperation

#### Fix The Genres Datatypes and Names

In [ ]:
def extract_genres(genre_str):
    try:
        genres = ast.literal_eval(genre_str)
        return [g['name'] for g in genres]
    except:
        return []

dataFrame = df[['index', 'title', 'Genres', 'release_date', 'Budget', 'Revenue', 'popularity', 'vote_average', 'vote_count']].copy()
dataFrame['genres'] = dataFrame['Genres'].apply(extract_genres)
dataFrame.drop('Genres', axis=1, inplace=True)

In [ ]:
dfGenreExploded = dataFrame.explode('genres')

#### Create A New DataFrame From The Columns That I Want To Use

In [ ]:
dataFrame = df[['index', 'title', 'Genres', 'release_date', 'Budget', 'Revenue', 'popularity', 'vote_average', 'vote_count']].copy()

dataFrame.rename(columns={
    'index': 'id',
    'title': 'title',
    'Genres': 'genres',
    'release_date': 'releaseDate',
    'Budget': 'budget',
    'Revenue': 'revenue',
    'popularity': 'popularity',
    'vote_average': 'rating',
    'vote_count': 'votes'
}, inplace=True)

#### Fix The Inappropriate Data Types

In [ ]:
dataFrame['releaseDate'] = pd.to_datetime(dataFrame['releaseDate']).dt.date

## Step 3: Feature Understanding

In [ ]:
dataFrame.head(5)

#### Count of Movies By Genres

In [ ]:
fig, ax = plt.subplots(figsize=(18, 10))

m = dfGenreExploded['genres'].values
n = dfGenreExploded['genres'].index


bars = sns.barplot(x= m, y= n)

chartStyling(ax=ax, fig=fig, title="Count of Movies By Genres", bars=bars.patches)

plt.show()

#### Distribution of Ratings

In [ ]:
ratingZeroToThree = dataFrame.query('rating >= 0 and rating <= 3').index
ratingFourToSix = dataFrame.query('rating > 3 and rating <= 6').index
ratingSevenToTen = dataFrame.query('rating > 6 and rating <= 10').index

values = [len(ratingZeroToThree), len(ratingFourToSix), len(ratingSevenToTen)]
labels = ['0-3', '4-6', '7-10']

pieChartStyling(values=values, labels=labels, title="Distribution of Ratings")

plt.show()

#### Distributon of Years

In [ ]:
date1890To1930 = dataFrame[dataFrame['releaseDate'] <= pd.to_datetime('1930-12-31').date()].index
date1930To1970 = dataFrame[(dataFrame['releaseDate'] > pd.to_datetime('1930-12-31').date()) & (dataFrame['releaseDate'] <= pd.to_datetime('1970-12-31').date())].index
date1970To2010 = dataFrame[(dataFrame['releaseDate'] > pd.to_datetime('1970-12-31').date()) & (dataFrame['releaseDate'] <= pd.to_datetime('2010-12-31').date())].index
date2010To2020 = dataFrame[dataFrame['releaseDate'] > pd.to_datetime('2010-12-31').date()].index

values = [len(date1890To1930), len(date1930To1970), len(date1970To2010), len(date2010To2020)]
labels = ['1890-1930', '1930-1970', '1970-2010', '2010-2020']

pieChartStyling(values=values, labels=labels, title="Distribution of Release Years")

plt.show()

#### Rating In Comparison to Budget


In [ ]:
fig, ax = plt.subplots(figsize=(18, 10))

sns.lineplot(x='rating', y='budget', data=dataFrame, ax=ax)

lineChartStyling(ax=ax, fig=fig, title="Rating In Comparison to Budget")

plt.show()

#### Rating In Comparison to Popularity

In [ ]:
fig, ax = plt.subplots(figsize=(18, 10))

bars = sns.barplot(x='rating', y='popularity', data=dataFrame, ax=ax)

chartStyling(ax=ax, fig=fig, title="Rating In Comparison to Popularity", bars=bars.patches)

plt.show()

#### Correlation Heatmap
I didn't take the 'revenue' column because values are exact same with the 'budget' column.

In [ ]:
numericColumns = ['budget', 'popularity', 'rating', 'votes']
correlation_matrix = dataFrame[numericColumns].corr()

In [ ]:
fig, ax = plt.subplots(figsize=(18, 10))

bars = sns.heatmap(correlation_matrix, annot= True)

chartStyling(ax=ax, fig=fig, title="Correlation Heatmap", bars=bars.patches)

plt.show()

#### Pair Plot

In [ ]:
topGenres = dfGenreExploded['genres'].value_counts().head(10).index
df_subset = dfGenreExploded[dfGenreExploded['genres'].isin(topGenres)]

numericColumns = ['Budget', 'Revenue', 'popularity', 'vote_average', 'vote_count']
sns.pairplot(df_subset, vars=numericColumns, hue='genres', palette='viridis', height=2.5)
plt.show()
